In [1]:
"""
import sys
!{sys.executable} -m pip install openpyxl
#"""

In [2]:
'''
CS 584: Applied BioNLP
Assignment 1: Evaluation script

The file simply loads a submission file and a gold standard and
computes the recall, precision and F-score for the system

@author Abeed Sarker
email: abeed.sarker@dbmi.emory.edu


Created: 09/14/2020
'''

In [7]:
import pandas as pd
from collections import defaultdict
from collections import Counter



def load_labels(f_path):
    '''
    Loads the labels

    :param f_path:
    :return:
    '''
    labeled_df = pd.read_excel(f_path)
    labeled_dict = defaultdict(list)
    for index,row in labeled_df.iterrows():
        id_ = row['ID']
        if not pd.isna(row['Symptom CUIs']) and not pd.isna(row['Negation Flag']):
            cuis = row['Symptom CUIs'].split('$$$')[1:-1]
            neg_flags = row['Negation Flag'].split('$$$')[1:-1]
            for cui,neg_flag in zip(cuis,neg_flags):
                labeled_dict[id_].append(cui + '-' + str(neg_flag))
    return labeled_dict


In [4]:
gold_standard_dict = load_labels('./Assignment1GoldStandardSet.xlsx')
#submission_dict = load_labels('./AssignmentSampleSubmission.xlsx')
submission_dict = load_labels('./TestGold8.xlsx')

In [5]:
#cui is not in otherCuis
#is this because the negation flag was wrong?

def isNegationError(cui, otherCuis):
    justCui=cui[:-2]
    for other in otherCuis:
        otherJustCui=other[:-2]
        if justCui==otherJustCui:
            return True
        
    return False

def isNegative(cui):
    flag=cui[-1]
    return flag=='1'



In [8]:
counts2=Counter()

for k,v in gold_standard_dict.items():
    for c in v:
        if isNegative(c):
            counts2['neg']+=1
        else:
            counts2['pos']+=1
            
counts2

Counter({'pos': 198, 'neg': 14})

In [9]:
from collections import Counter
counts=Counter()

tp = 0
tn = 0
fp = 0
fn = 0
for k,v in gold_standard_dict.items():
    for c in v:
        try:
            if c in submission_dict[k]:
                tp+=1
            else:
                #print(c, submission_dict[k])
                if isNegationError(c, submission_dict[k]):
                    #print("is negation error")
                    counts['fnneg']+=1
                    if isNegative(c):
                        counts['notnegenogh']+=1
                    else:
                        counts['too neg']+=1
                else:
                    #print("is content error")
                    counts['fncont']+=1
                fn+=1
        except KeyError:#if the key is not found in the submission file, each is considered
                        #to be a false negative..
            fn+=1
    for c2 in submission_dict[k]:
        if not c2 in gold_standard_dict[k]:
            fp+=1
            if isNegationError(c2, gold_standard_dict[k]):
                    #print("is negation error")
                    counts['fpneg']+=1
                    if not isNegative(c2):
                        counts['notnegenogh']+=1
                    else:
                        counts['too neg']+=1
            else:
                    #print("is content error")
                    counts['fpcont']+=1
print('True Positives:',tp, 'False Positives: ', fp, 'False Negatives:', fn)
recall = tp/(tp+fn)
precision = tp/(tp+fp)
f1 = (2*recall*precision)/(recall+precision)
print('Recall: ',recall,'\nPrecision:',precision,'\nF1-Score:',f1)
print(counts)



True Positives: 151 False Positives:  84 False Negatives: 61
Recall:  0.7122641509433962 
Precision: 0.6425531914893617 
F1-Score: 0.6756152125279643
Counter({'fpcont': 76, 'fncont': 46, 'too neg': 20, 'fnneg': 15, 'fpneg': 8, 'notnegenogh': 3})
